In [7]:
import numpy as np
import mne
from mne.preprocessing import ICA
import sklearn
import matplotlib.pyplot as plt
import pywt

In [2]:
# 분석대상 이름 입력
name = input("분석대상 이름을 입력하세요.")
print(f"분석대상 이름은 {name}입니다.")

# 분석 목적 입력
purpose = input("분석 목적을 입력하세요. load와 append 중 입력하세요.")
print(f"분석 목적은 {purpose}입니다.")

# load 선택 시
if purpose == 'load':
    print("load를 선택하였습니다.")
    # Specify the full path to EEG data file (EDF format)
    eeg_file_path = rf'H:\Mg_EEG\edf_files\{name}_7200.EDF'
    
    # Load EEG data
    eeg_data = mne.io.read_raw_edf(eeg_file_path, preload=True)
    print(f"성공적으로 데이터를 불러왔습니다.")

# append 선택 시
elif purpose == 'append':
    print("append를 선택하였습니다.")
    # Specify the full path to EEG data file (EDF format)
    eeg_file_path_before = rf'H:\Mg_EEG\edf_files\{name}_before_3600.EDF'
    eeg_file_path_after = rf'H:\Mg_EEG\edf_files\{name}_after_3600.EDF'
    
    # Load EEG data
    eeg_data_before = mne.io.read_raw_edf(eeg_file_path_before, preload=True)
    eeg_data_after = mne.io.read_raw_edf(eeg_file_path_after, preload=True)
    print(f"성공적으로 데이터를 불러왔습니다.")

    # 3600초 이상의 데이터는 3600초로 자르기
    for data in [eeg_data_before, eeg_data_after]:
        if len(data) > 3600*200:
            data.crop(tmax=3600, include_tmax=False)
            print(f"{data.filenames[0]}: 데이터 길이가 3600초 이상이므로 3600초로 자름.")
        else:
            print(f"{data.filenames[0]}:데이터 길이가 3600초 미만이므로 그대로 사용.")
    
    # 데이터 합치기
    eeg_data = eeg_data_before.copy()
    eeg_data.append(eeg_data_after)
    if len(eeg_data) == 7200*200:
        print("데이터가 성공적으로 합쳐졌습니다.")
    else:
        print("문제가 발생하였습니다. 확인 바랍니다.")
else:
    print("올바른 값을 입력하세요.")

분석대상 이름은 김영현입니다.
분석 목적은 load입니다.
load를 선택하였습니다.
Extracting EDF parameters from H:\Mg_EEG\edf_files\김영현_7200.EDF...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 1439999  =      0.000 ...  7199.995 secs...
성공적으로 데이터를 불러왔습니다.


In [3]:
# Specify the time range of interest
start_time = 0  # Start time in seconds
duration = 7200 # 2 hours in seconds
end_time = start_time + duration # End time in seconds (2 hours later from start_time)
# Mark the time of magnesium infusion
time_of_infusion = 3600  # Time of infusion in seconds

# Crop EEG data
eeg_data_window = eeg_data.copy().crop(tmin=start_time, tmax=7199)

# Notch filter apply
eeg_data_window.notch_filter(freqs=[60])

# Bandpass filters
eeg_data_window.filter(l_freq=0.1, h_freq=30)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 59.35
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 59.10 Hz)
- Upper passband edge: 60.65 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 60.90 Hz)
- Filter length: 1321 samples (6.605 s)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband ed

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.5s


Measurement date,"April 11, 2015 01:25:00 GMT"
Experimenter,Unknown
Participant,No
Digitized points,Not available
Good channels,22 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,200.00 Hz
Highpass,0.10 Hz
Lowpass,30.00 Hz


In [29]:
# Step 3: 웨이블릿 변환 수행
# PyWavelets를 사용하여 웨이블릿 변환을 수행합니다.
n_channels, n_times = eeg_data_window._data.shape
wavelet_data = np.zeros_like(eeg_data_window._data)

wavelet = 'db4'  # Daubechies 4번 웨이블릿 사용
level = 4  # 웨이블릿 분해 레벨

for ch in range(n_channels):
    # DWT를 적용하여 계수를 계산합니다.
    coeffs = pywt.wavedec(eeg_data_window._data[ch, :], wavelet, level=level)

    # 특정 수준의 근사 계수만을 사용하여 신호를 재구성합니다.
    cA = coeffs[0]
    rec_data = pywt.waverec([cA] + [None] * (level - 1), wavelet)

    # 재구성된 데이터가 원본 길이보다 길거나 짧은 경우 처리
    if len(rec_data) > n_times:
        wavelet_data[ch, :] = rec_data[:n_times]  # 길이가 길다면 잘라줌
    else:
        wavelet_data[ch, :len(rec_data)] = rec_data  # 길이가 짧다면 할당

In [30]:
# Step 4: ICA 적용을 위해 웨이블릿 변환된 데이터를 새로운 MNE 객체로 생성
# Wavelet 변환된 데이터를 이용해 Raw 객체 생성
info = eeg_data_window.info  # 기존 정보 복사
wavelet_raw = mne.io.RawArray(wavelet_data, info)

Creating RawArray with float64 data, n_channels=22, n_times=1439801
    Range : 0 ... 1439800 =      0.000 ...  7199.000 secs
Ready.


In [31]:
wavelet_raw.ch_names

['Fp1-F3',
 'F3-C3',
 'C3-P3',
 'P3-O1',
 'Fp2-F4',
 'F4-C4',
 'C4-P4',
 'P4-O2',
 'Fp1-F7',
 'F7-T3',
 'T3-T5',
 'T5-O1',
 'Fp2-F8',
 'F8-T4',
 'T4-T6',
 'T6-O2',
 'FZ-CZ',
 'CZ-PZ',
 'EKG',
 'EOGLt-A1',
 'EOGRt-A2',
 'Photic']

In [36]:
wavelet_data_trans = wavelet_raw.copy()

wavelet_data_trans.drop_channels(wavelet_raw.ch_names[18:])

wavelet_data_trans_ch_names = wavelet_data_trans.ch_names

ch_names_answer = ['Fp1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 
                'Fp2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 
                'Fp1-F7', 'F7-T3', 'T3-T5', 'T5-O1', 
                'Fp2-F8', 'F8-T4', 'T4-T6', 'T6-O2', 
                'Fz-Cz', 'Cz-Pz']

# 차이점을 저장할 딕셔너리 초기화
channel_mapping = {}

# 리스트 길이만큼 반복
for i in range(len(wavelet_data_trans_ch_names)):
    if eeg_data_trans_ch_names[i] != ch_names_answer[i]:
        # 차이가 있는 경우, 해당 인덱스의 값을 딕셔너리에 저장
        channel_mapping[eeg_data_trans_ch_names[i]] = ch_names_answer[i]

# 결과 출력
print(channel_mapping)

{'FZ-CZ': 'Fz-Cz', 'CZ-PZ': 'Cz-Pz'}


In [37]:
wavelet_data_trans.rename_channels(channel_mapping)

Measurement date,"April 11, 2015 01:25:00 GMT"
Experimenter,Unknown
Participant,No
Digitized points,Not available
Good channels,18 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,200.00 Hz
Highpass,0.10 Hz
Lowpass,30.00 Hz
